<a href="https://colab.research.google.com/github/jnhutton/home_sales/blob/main/Home_Sales_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,552 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 1,781 kB in 1s (1,244 kB/s)



In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
home_sale_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

home_sale_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Create a temporary view of the DataFrame.
home_sale_df.createOrReplaceTempView('home_sales')

In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""
SELECT year(date), bedrooms, round(avg(price),2) as AveragePrice
FROM home_sales
WHERE bedrooms=4
GROUP BY year(date), bedrooms
""").show()

+----------+--------+------------+
|year(date)|bedrooms|AveragePrice|
+----------+--------+------------+
|      2020|       4|   298353.78|
|      2019|       4|    300263.7|
|      2022|       4|   296363.88|
|      2021|       4|   301819.44|
+----------+--------+------------+



In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""
SELECT year(date_built), bedrooms, bathrooms, round(avg(price),2) as AveragePrice
FROM home_sales
WHERE (bedrooms=3 and bathrooms=3)
GROUP BY year(date_built), bedrooms, bathrooms
""").show()

+----------------+--------+---------+------------+
|year(date_built)|bedrooms|bathrooms|AveragePrice|
+----------------+--------+---------+------------+
|            2015|       3|        3|    288770.3|
|            2011|       3|        3|   291117.47|
|            2013|       3|        3|   295962.27|
|            2014|       3|        3|   290852.27|
|            2012|       3|        3|   293683.19|
|            2010|       3|        3|   292859.62|
|            2017|       3|        3|   292676.79|
|            2016|       3|        3|   290555.07|
+----------------+--------+---------+------------+



In [7]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""
SELECT year(date_built), bedrooms, bathrooms, floors, sqft_living, round(avg(price),2) as AveragePrice
FROM home_sales
WHERE (bedrooms=3 and bathrooms=3 and floors=2 and sqft_living >= 2000)
GROUP BY year(date_built), bedrooms, bathrooms, floors, sqft_living
""").show()

+----------------+--------+---------+------+-----------+------------+
|year(date_built)|bedrooms|bathrooms|floors|sqft_living|AveragePrice|
+----------------+--------+---------+------+-----------+------------+
|            2010|       3|        3|     2|       2236|    438333.0|
|            2013|       3|        3|     2|       2431|    264259.5|
|            2011|       3|        3|     2|       2079|    249107.0|
|            2016|       3|        3|     2|       2355|    258723.0|
|            2015|       3|        3|     2|       2030|    390345.0|
|            2014|       3|        3|     2|       2289|    259821.0|
|            2016|       3|        3|     2|       2330|    141841.0|
|            2017|       3|        3|     2|       2353|    149685.0|
|            2016|       3|        3|     2|       2274|    406129.0|
|            2014|       3|        3|     2|       2191|   303204.67|
|            2015|       3|        3|     2|       2212|    267851.0|
|            2014|  

In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
SELECT view, round(avg(price),2)
FROM home_sales
GROUP BY view
HAVING (avg(price) >= 350000)
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  51|           788128.21|
|  54|           798684.82|
|  69|           750537.94|
|  87|           1072285.2|
|  73|           752861.18|
|  64|           767036.67|
|  59|            791453.0|
|  85|          1056336.74|
|  52|           733780.26|
|  71|            775651.1|
|  98|          1053739.33|
|  99|          1061201.42|
|  96|          1017815.92|
| 100|           1026669.5|
|  70|           695865.58|
|  61|           746877.59|
|  75|          1114042.94|
|  78|          1080649.37|
|  89|          1107839.15|
|  77|          1076205.56|
+----+--------------------+
only showing top 20 rows

--- 0.9299571514129639 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [11]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""
SELECT view, round(avg(price),2)
FROM home_sales
GROUP BY view
HAVING (avg(price) >= 350000)
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  51|           788128.21|
|  54|           798684.82|
|  69|           750537.94|
|  87|           1072285.2|
|  73|           752861.18|
|  64|           767036.67|
|  59|            791453.0|
|  85|          1056336.74|
|  52|           733780.26|
|  71|            775651.1|
|  98|          1053739.33|
|  99|          1061201.42|
|  96|          1017815.92|
| 100|           1026669.5|
|  70|           695865.58|
|  61|           746877.59|
|  75|          1114042.94|
|  78|          1080649.37|
|  89|          1107839.15|
|  77|          1076205.56|
+----+--------------------+
only showing top 20 rows

--- 0.5091774463653564 seconds ---


In [12]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sale_df.write.partitionBy("date_built").parquet("home_sales_partitioned", mode="overwrite")

In [13]:
# 11. Read the parquet formatted data.
p_home_sale_df_p=spark.read.parquet('home_sales_partitioned')

In [14]:
# 12. Create a temporary table for the parquet data.
p_home_sale_df_p.createOrReplaceTempView('p_home_sales_p')

In [15]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""
SELECT view, round(avg(price),2)
FROM p_home_sales_p
GROUP BY view
HAVING (avg(price) >= 350000)
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  51|           788128.21|
|  54|           798684.82|
|  69|           750537.94|
|  73|           752861.18|
|  87|           1072285.2|
|  64|           767036.67|
|  59|            791453.0|
|  85|          1056336.74|
|  52|           733780.26|
|  71|            775651.1|
|  98|          1053739.33|
|  99|          1061201.42|
|  96|          1017815.92|
| 100|           1026669.5|
|  70|           695865.58|
|  61|           746877.59|
|  75|          1114042.94|
|  78|          1080649.37|
|  89|          1107839.15|
|  77|          1076205.56|
+----+--------------------+
only showing top 20 rows

--- 0.7935571670532227 seconds ---


In [16]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [17]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached("home_sales"):
  print("the table is till cached")
else:
  print("all clear")

all clear
